In [ ]:
import argparse
import time
import shutil
import os
import os.path as osp
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
import torchvision.models as models
from resnext_specialist import VA
from data_cnn60 import NTUDataLoaders, AverageMeter, make_dir, get_cases, get_num_classes
from sklearn.metrics import confusion_matrix
from collections import OrderedDict
import torch.nn.functional as F
from fgar import AE, PosMmen, JPosMmen, contractive_loss, MMDLoss, multi_class_hinge_loss, triplet_loss 

# parser = argparse.ArgumentParser(description='View adaptive')
# parser.add_argument('--ss', type=int, help="split size")
# parser.add_argument('--st', type=str, help="split type")
# parser.add_argument('--dataset', type=str, help="dataset path")
# parser.add_argument('--wdir', type=str, help="directory to save weights path")
# parser.add_argument('--le', type=str, help="language embedding model")
# parser.add_argument('--ve', type=str, help="visual embedding model")
# parser.add_argument('--phase', type=str, help="train or val")
# parser.add_argument('--gpu', type=str, help="gpu device number")
# args = parser.parse_args()

gpu = '0'
ss = 12
st = 'r'
dataset_path = 'ntu_results/shift_12_r_1'
# wdir = gating_our
le = 'bert_large'
ve = 'shift'
phase = 'train'
num_classes = 60

os.environ["CUDA_VISIBLE_DEVICES"] = gpu
seed = 5
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
device = torch.device("cuda")
print(torch.cuda.device_count())


# if not os.path.exists('/ssd_scratch/cvit/pranay.gupta/language_modelling/' + wdir):
#     os.mkdir('/ssd_scratch/cvit/pranay.gupta/language_modelling/' + wdir)
# if not os.path.exists('/ssd_scratch/cvit/pranay.gupta/language_modelling/' + wdir + '/' + le):
#     os.mkdir('/ssd_scratch/cvit/pranay.gupta/language_modelling/' + wdir + '/' + le)

# trunk_checkpoint = '/ssd_scratch/cvit/pranay.gupta/ntu_results/vacnn_val_'+str(ss)+'_'+st+'/best_val_0.pth.tar'
# trunk = nn.DataParallel(VA(60), device_ids=[0, 1, 2, 3])
# trunk = trunk.to(device)
# criterion1 = nn.CrossEntropyLoss(reduction = 'none').to(device)
# trunk.load_state_dict(torch.load(trunk_checkpoint)['trunk_state_dict'], strict=False)
# trunk_optimizer = optim.Adam(trunk.parameters(), lr=0.00001, weight_decay = 0.001)
# trunk_scheduler = ReduceLROnPlateau(trunk_optimizer, mode='max', factor=0.1, patience=2, cooldown=2, verbose=True)
# print("Loaded Visual Encoder")


criterion2 = nn.MSELoss().to(device)

if ve == 'vacnn':
    vis_emb_input_size = 2048
elif ve == 'shift':
    vis_emb_input_size = 256
else: 
    pass    
    
text_hidden_size = 100
vis_hidden_size = 512
output_size = 50

if le == 'bert_large':
    noun_emb_input_size = 1024
    verb_emb_input_size = 1024
elif le == 'w2v':
    noun_emb_input_size = 300
    verb_emb_input_size = 300
else:
    pass

ntu_loaders = NTUDataLoaders(dataset_path, 'max', 1)
train_loader = ntu_loaders.get_train_loader(1024, 8)
zsl_loader = ntu_loaders.get_val_loader(1024, 8)
val_loader = ntu_loaders.get_test_loader(1024, 8)
zsl_out_loader = ntu_loaders.get_val_out_loader(1024, 8)
val_out_loader = ntu_loaders.get_test_out_loader(1024, 8)
train_size = ntu_loaders.get_train_size()
zsl_size = ntu_loaders.get_val_size()
val_size = ntu_loaders.get_test_size()
print('Train on %d samples, validate on %d samples' % (train_size, zsl_size))



nouns_vocab = np.load('nouns_vocab.npy')
verbs_vocab = np.load('verbs_vocab.npy')
nouns = nouns_vocab[np.argmax(np.load('nouns_ohe.npy'), -1)][:num_classes]
# nouns[nouns == 'object'] = '#'
# nouns[51] = 'someone'
# nouns[55] = 'object'
# nouns = np.load('nouns.npy')
verbs = verbs_vocab[np.argmax(np.load('verbs_ohe.npy'), -1)][:num_classes]
# nouns = np.load('nouns.npy')
# verbs = np.load('verbs.npy')
# prps = np.load('prepositions.npy')
labels = np.load('labels.npy')

if phase == 'val':
    gzsl_inds = np.load('./label_splits/'+ st + 's' + str(num_classes - ss) +'.npy')
    unseen_inds = np.sort(np.load('./label_splits/' + st + 'v' + str(ss) + '_0.npy'))
    seen_inds = np.load('./label_splits/'+ st + 's' + str(num_classes - ss -ss) + '_0.npy')
else:
    gzsl_inds = np.arange(num_classes)
    unseen_inds = np.sort(np.load('./label_splits/' + st + 'u' + str(ss) + '_1.npy'))
    seen_inds = np.load('./label_splits/'+ st + 's' + str(num_classes - ss) + '_1.npy')

unseen_labels = labels[unseen_inds]
seen_labels = labels[seen_inds]

unseen_nouns = nouns[unseen_inds]
unseen_verbs = verbs[unseen_inds]
# unseen_prps = prps[unseen_inds]
seen_nouns = nouns[seen_inds]
seen_verbs = verbs[seen_inds]
# seen_prps = prps[seen_inds]
verb_corp = np.unique(verbs[gzsl_inds])
noun_corp = np.unique(nouns[gzsl_inds])
# prp_corp = np.unique(prps[gzsl_inds])

# import gensim
# model = gensim.models.KeyedVectors.load_word2vec_format('/ssd_scratch/cvit/pranay.gupta/GoogleNews-vectors-negative300.bin', binary=True)


# def get_w2v(model, words):
#     emb = np.zeros([300])
#     for word in words.split():
#         emb += model[word]
#     emb /= len(words.split())
    
#     return emb


verb_emb = torch.from_numpy(np.load(le + '_verb.npy')[:num_classes, :]).view([num_classes, verb_emb_input_size])
verb_emb = verb_emb/torch.norm(verb_emb, dim = 1).view([num_classes, 1]).repeat([1, verb_emb_input_size])
noun_emb = torch.from_numpy(np.load(le + '_noun.npy')[:num_classes, :]).view([num_classes, noun_emb_input_size])
noun_emb = noun_emb/torch.norm(noun_emb, dim = 1).view([num_classes, 1]).repeat([1, noun_emb_input_size])
# prp_w2v = torch.from_numpy(np.array([get_w2v(model, i) for i in prps])).view([60, 300])
# prp_w2v = noun_emb/torch.norm(prp_w2v, dim = 1).view([60, 1]).repeat([1, 300])

unseen_verb_emb = verb_emb[unseen_inds, :]
unseen_noun_emb = noun_emb[unseen_inds, :]
# unseen_prp_w2v = prp_w2v[unseen_inds, :]

seen_verb_emb = verb_emb[seen_inds, :]
seen_noun_emb = noun_emb[seen_inds, :]
# seen_prp_w2v = prp_w2v[seen_inds, :]
print("loaded language embeddings")


def get_text_data(target, verb_emb, noun_emb):
    return verb_emb[target].view(target.shape[0], verb_emb_input_size).float(), noun_emb[target].view(target.shape[0], verb_emb_input_size).float()


def save_checkpoint(state, filename='checkpoint.pth.tar', is_best=False):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')
        
def accuracy(class_embedding, vis_trans_out, target, inds):
    inds = torch.from_numpy(inds).to(device)
    temp_vis = vis_trans_out.unsqueeze(1).expand(vis_trans_out.shape[0], class_embedding.shape[0], vis_trans_out.shape[1])
    temp_cemb = class_embedding.unsqueeze(0).expand(vis_trans_out.shape[0], class_embedding.shape[0], vis_trans_out.shape[1])
    preds = torch.argmax(torch.sum(temp_vis*temp_cemb, axis=2), axis = 1)
    acc = torch.sum(inds[preds] == target).item()/(preds.shape[0])
    return acc, torch.sum(temp_vis*temp_cemb, axis=2)

In [ ]:
tars = []
for i, (_, target) in enumerate(val_loader):
    tars += list(target.numpy())

In [1]:
import numpy as np

In [2]:
tars = np.load('../../synse_resources/ntu_results/shift_12_r/g_label.npy')

In [3]:
unseen_inds = np.load('../../synse_resources/resources/label_splits/ru12.npy')

In [4]:
seen_inds = np.load('../../synse_resources/resources/label_splits/rs48.npy')

In [5]:
test_y = []
for i in tars:
    if i in unseen_inds:
        test_y.append(0)
    else:
        test_y.append(1)

In [6]:
import numpy as np
test_zs = np.load('../../tf_vaegan_test/12_12_r_gzsl_zs (1).npy')
# np.array([j.cpu().detach().numpy() for i in test_zs for j in i])
# test_noun_zs = np.array([j.cpu().detach().numpy() for i in test_noun_zs for j in i])
# test_verb_zs = np.array([j.cpu().detach().numpy() for i in test_verb_zs for j in i])
# test_seen = np.array([j.cpu().detach().numpy() for i in test_seen for j in i])
test_seen = np.load('../../synse_resources/ntu_results/shift_12_r/gtest_out.npy')

In [7]:
test_zs[0]

array([1.8229499e-03, 6.6124124e-04, 1.0512970e-05, 3.4663008e-04,
       1.7086327e-03, 9.6961075e-01, 2.9532635e-03, 1.3373049e-02,
       1.7795999e-06, 2.4276970e-04, 9.1789868e-03, 8.9412082e-05],
      dtype=float32)

In [8]:
test_seen[0]

array([16.497099  ,  4.4249506 ,  9.405852  , -1.8997799 , -1.4513168 ,
       -1.8997767 , -3.175321  ,  0.4946652 , -0.196605  , -1.8997812 ,
        1.6111653 ,  1.7553184 , -1.8997788 ,  1.1744123 , -0.34749636,
       -1.8997806 , -0.41130373,  5.441618  , -3.8300982 ,  1.1667404 ,
       -0.44223854, -2.403778  , -2.1648352 , -2.2025666 ,  0.3654217 ,
       -2.5834043 , -2.691006  ,  6.1159782 ,  1.4461741 , -1.8221828 ,
       -4.0724807 ,  5.739887  ,  2.1140084 , -3.2194986 ,  0.04018287,
       -1.1460462 , -0.24068251,  2.9343874 , -0.38375956,  0.08082317,
       -1.8997825 , -0.6741345 , -1.899781  ,  5.8188105 , -1.6423881 ,
        2.7963684 ,  2.3555338 , -1.899781  , -1.6626861 , -3.4799366 ,
       -3.177768  , -1.8997792 , -1.568146  , -2.856479  , -0.26817882,
        0.7377131 , -1.8997791 , -1.6005936 , -1.8997856 , -1.8997767 ],
      dtype=float32)

In [9]:
test_seen.shape
# test_zs.shape

(16392, 60)

In [10]:
def temp_scale(seen_features, T):
    return np.array([np.exp(i)/np.sum(np.exp(i)) for i in (seen_features + 1e-12)/T])

In [11]:
prob_test_zs = test_zs
# np.array([np.exp(i)/np.sum(np.exp(i)) for i in test_zs])
# prob_test_noun_zs = np.array([np.exp(i)/np.sum(np.exp(i)) for i in test_noun_zs])
# prob_test_verb_zs = np.array([np.exp(i)/np.sum(np.exp(i)) for i in test_verb_zs])
prob_test_seen = temp_scale(test_seen, 4)
# prob_test_seen = np.array([np.exp(i)/np.sum(np.exp(i)) for i in test_seen])

In [12]:
feat_test_zs = np.sort(prob_test_zs, 1)[:,::-1][:,:12]
# feat_test_noun_zs = np.sort(prob_test_noun_zs, 1)[:,::-1]
# feat_test_verb_zs = np.sort(prob_test_verb_zs, 1)[:,::-1]
feat_test_seen = np.sort(prob_test_seen, 1)[:,::-1][:,:12]

In [13]:
# mod_feat_test_zs = np.concatenate([feat_test_noun_zs, feat_test_verb_zs], 1)
gating_test_x = np.concatenate([feat_test_zs, feat_test_seen], 1)
gating_test_y = test_y
# gating_train_x = np.concatenate([np.concatenate([feat_unseen_zs[train_unseen_inds, :], feat_unseen_train[train_unseen_inds, :]], 1), np.concatenate([feat_seen_zs[train_seen_inds, :], feat_seen_train[train_seen_inds, :]], 1)], 0)
# gating_train_y = [0]*len(train_unseen_inds) + [1]*len(train_seen_inds)

In [14]:
len(gating_test_y)

16392

In [15]:
import pickle as pkl
with open('../../tf_vaegan_test/gating_model_t4_thresh0.74_seen.pkl', 'rb') as f:
    gating_model = pkl.load(f)

In [16]:
prob_gate = gating_model.predict_proba(gating_test_x)

In [17]:
pred_test = 1 - prob_gate[:, 0]>0.50

In [18]:
# pred_val = gating_model.predict(gating_val_x)
np.sum(pred_test == test_y)/len(test_y)

0.4874328940946803

In [19]:
a = prob_gate
b = np.zeros(prob_gate.shape[0])

In [20]:
p_gate_seen = prob_gate[:, 1]
prob_y_given_seen = prob_test_seen + (1/60)*np.repeat((1 - p_gate_seen)[:, np.newaxis], 60, 1)

In [21]:
p_gate_unseen = prob_gate[:, 0]
prob_y_given_unseen = prob_test_zs + (1/12)*np.repeat((1 - p_gate_unseen)[:, np.newaxis], 12, 1)

In [22]:
prob_seen = prob_y_given_seen*np.repeat(p_gate_seen[:, np.newaxis], 60, 1)
prob_unseen = prob_y_given_unseen*np.repeat(p_gate_unseen[:, np.newaxis], 12, 1)

In [23]:
unseen_inds = sorted(unseen_inds)

In [24]:
final_preds = []

# soft gating
# for i in range(len(prob_seen)):
#     if np.max(prob_seen[i, :]) > np.max(prob_unseen[i, :]):
#         pred = seen_inds[np.argmax(prob_seen[i, :])]
#     else:
#         pred = unseen_inds[np.argmax(prob_unseen[i, :])]
#     final_preds.append(pred)

# hard gating
# final_preds = seen_inds[np.argmax(prob_test_seen, -1)]*pred_test + unseen_inds[np.argmax(prob_test_zs, -1)]*(1-pred_test)
seen_count = 0
tot_seen = 0
unseen_count = 0
tot_unseen = 0
gseen_count = 0
gunseen_count = 0
for i in range(len(gating_test_y)):
    if pred_test[i] == 1:
        pred = seen_inds[np.argmax(prob_test_seen[i, seen_inds])]
    else:
        pred = unseen_inds[np.argmax(prob_test_zs[i, :])]
    
    if tars[i] in seen_inds:
        tot_seen += 1
        if pred_test[i] == 1:
            gseen_count += 1
        if pred == tars[i]:
            seen_count += 1
    else:
        if pred_test[i] == 0:
            gunseen_count+=1
        tot_unseen += 1
        if pred == tars[i]:
            unseen_count += 1
    final_preds.append(pred)


In [25]:
gseen_count/tot_seen

0.3766194177716811

In [26]:
gunseen_count/tot_unseen

0.9321100917431193

In [27]:
seen_acc = seen_count/tot_seen

In [28]:
seen_acc

0.37486663618350863

In [29]:
unseen_acc = unseen_count/tot_unseen

In [30]:
unseen_acc

0.26238532110091745

In [31]:
h_mean = 2*seen_acc*unseen_acc/(seen_acc + unseen_acc)

In [32]:
h_mean

0.3086989426417084

In [36]:
unseen_inds

[3, 5, 9, 12, 15, 40, 42, 47, 51, 56, 58, 59]

In [37]:
unseen_count

227

In [38]:
tot_unseen

3270

In [39]:
tot_seen

13122

In [ ]:
len(tars)

In [ ]:
np.load('/ssd_scratch/cvit/pranay.gupta/ntu_results/shift_10_r/g_label.npy')

In [ ]:
uc = 0
for n in tars:
    if n in unseen_inds:
        uc+=1

In [ ]:
uc

In [ ]:
len(seen_inds)

In [ ]:
#softgating

In [23]:
prob_seen[:,unseen_inds] = prob_unseen
final_prob = prob_seen
final_pred = np.argmax(final_prob, -1)

In [24]:
seen_count = 0
seen_hit = 0
unseen_count = 0
unseen_hit = 0
gating_seen_hit = 0
gating_unseen_hit = 0
for i, gt in enumerate(tars):
    if gt in seen_inds:
#         if prob_gate[i, 0] < prob_gate[i, 1]:
#             gating_seen_hit += 1
        seen_count += 1
        if final_pred[i] == gt:
            seen_hit += 1
    else:
        if prob_gate[i, 0] > prob_gate[i, 1]:
            gating_unseen_hit += 1
        unseen_count += 1
        if final_pred[i] == gt:
            unseen_hit += 1

seen_acc = seen_hit/seen_count
unseen_acc = unseen_hit/unseen_count
h_mean = 2*seen_acc*unseen_acc/(seen_acc + unseen_acc)

In [25]:
seen_acc

0.586114921505868

In [26]:
unseen_acc

0.07186544342507645

In [27]:
h_mean

0.12803241852511435

In [28]:
seen_acc

0.586114921505868

In [29]:
unseen_acc

0.07186544342507645

In [30]:
h_mean

0.12803241852511435

In [33]:
a = np.load('../../tf_vaegan_test/12_12_r_gzsl_zs.npy')
b = np.load('../../synse_resources/ntu_results/shift_12_r/g_label.npy')

In [34]:
a.shape

(16392, 12)

In [35]:
b.shape

(16392,)

In [37]:
gzsl_out = np.argmax(a, -1)

In [50]:
for i in range(len(b)):
    if b[i] in unseen_inds:
        print(unseen_inds_old[gzsl_out[i]], b[i])

47 3
47 5
15 9
15 12
40 15
51 40
40 42
12 47
47 51
3 56
15 58
15 59
40 3
12 5
47 9
9 12
9 15
42 40
51 42
15 47
40 51
47 56
58 58
47 59
3 3
47 5
3 9
51 12
59 15
12 40
3 42
3 47
12 51
3 56
59 58
9 59
12 3
5 5
42 9
59 12
47 15
12 40
9 42
59 47
59 51
58 56
42 58
12 59
47 3
47 5
15 9
15 12
40 15
12 40
56 42
12 47
42 51
59 56
40 58
47 59
40 3
12 5
59 9
59 12
3 15
59 40
56 42
15 47
40 51
47 56
58 58
59 59
9 3
9 5
58 9
9 12
59 15
5 40
47 42
5 47
9 51
12 56
9 58
3 59
40 3
58 5
12 9
9 12
47 15
9 40
56 42
59 47
40 51
42 56
51 58
58 59
42 3
56 5
15 9
40 12
51 15
12 40
56 42
12 47
40 51
40 56
15 58
40 59
51 3
58 5
3 9
9 12
9 15
59 40
51 42
40 47
12 51
42 56
47 58
5 59
47 3
9 5
47 9
59 12
40 15
15 40
47 42
42 47
5 51
47 56
59 58
59 59
40 3
58 5
56 9
9 12
42 15
9 40
58 42
9 47
12 51
40 56
12 58
51 59
42 3
56 5
40 9
58 12
59 15
40 40
58 42
51 47
42 51
15 56
15 58
15 59
51 3
5 5
42 9
5 12
12 15
42 40
56 42
5 47
40 51
59 56
58 58
3 59
5 3
47 5
58 9
3 15
40 40
12 42
58 47
42 51
9 56
3 58
12 59
3 3
51 5
5

12 42
3 47
59 51
58 58
58 59
51 5
42 9
56 12
59 15
9 40
51 42
3 47
58 51
42 56
51 58
9 59
47 3
59 5
15 9
15 12
12 15
56 40
58 42
9 47
47 51
40 56
15 58
40 59
12 3
3 5
3 9
3 12
9 15
3 40
15 42
15 47
5 51
3 56
5 58
42 59
5 3
3 5
56 9
12 12
40 15
3 40
51 42
47 47
3 51
9 56
12 58
40 59
56 3
12 5
42 9
47 12
51 15
58 40
51 42
12 47
58 51
9 56
42 58
42 59
51 3
40 5
15 9
12 12
59 15
56 40
12 42
42 47
51 51
40 56
40 58
51 59
47 3
3 5
9 9
9 12
12 15
15 40
15 42
51 47
51 51
5 56
3 58
12 59
12 3
12 5
3 9
42 12
40 15
47 40
3 42
12 47
12 51
56 56
59 58
51 59
58 3
42 5
9 9
15 12
3 15
56 40
51 42
56 47
9 51
56 56
51 59
15 3
15 5
5 9
47 12
3 15
58 40
42 42
59 47
15 51
12 56
3 58
59 59
3 3
9 5
9 9
3 12
42 15
58 40
40 42
59 47
9 51
9 56
59 58
51 59
59 3
56 5
56 9
56 12
51 15
5 40
3 42
47 47
47 51
51 56
56 58
56 59
12 3
59 5
47 9
59 12
15 15
47 40
3 42
56 47
58 51
58 56
47 58
56 59
40 3
15 5
59 9
3 12
58 15
51 40
42 42
51 47
58 51
12 56
3 58
47 59
56 3
12 5
59 9
15 12
59 15
51 40
5 42
51 47
3 51
51 56
12 

In [44]:
unseen_inds

[3, 5, 9, 12, 15, 40, 42, 47, 51, 56, 58, 59]

In [45]:
unseen_inds_old = np.load('../../synse_resources/resources/label_splits/ru12.npy')

In [46]:
unseen_inds_old

array([ 9, 42, 12, 58, 15, 51, 47, 59,  5,  3, 56, 40])

In [47]:
85*39

3315